In [ ]:
import folium
import pandas as pd
# import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
map1 = folium.Map(location=[37.566535,126.97796919999996],zoom_start=8,
                 zoom_control=False, tiles='https://tiles.stadiamaps.com/tiles/outdoors/{z}/{x}/{y}{r}.png',
                 attr='&copy; <a href="https://stadiamaps.com/">Stadia Maps</a>, &copy; <a href="https://openmaptiles.org/">OpenMapTiles</a> &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors')
#map1

#### for colab

### 관광지 marker cluster

In [ ]:
from folium.plugins import MarkerCluster
mc = MarkerCluster().add_to(map1)

In [ ]:
data2 = data[['관광지명','위도','경도']]
data2

In [ ]:
data2 = data2.dropna()
data2

In [ ]:
leng = len(data2)

In [ ]:
for i in range(leng):
    folium.Marker([data2.iloc[i][1],data2.iloc[i][2]],popup=data2.iloc[i][0]).add_to(mc)

In [ ]:
map1

# Folium 지역 경계선 구분하기

In [ ]:
#skorea-geo.json
#skorea-municipalities-geo
#skorea-provinces-geo

In [ ]:
import json

### 전국 하나로

In [ ]:
with open('../data/skorea-geo.json',encoding = 'euc-kr') as file:
    geo = file.read()
    geo = json.loads(geo)
    file.close()

In [ ]:
geo

In [ ]:
folium.GeoJson(geo).add_to(map1)
map1

### 시, 동 단위 세부

In [ ]:
with open('data/skorea-municipalities-geo.json',encoding = 'euc-kr') as file:
    geo2 = file.read()
    geo2 = json.loads(geo2)
    file.close()

In [ ]:
folium.GeoJson(geo2).add_to(map1)
map1

### 17개 광역시, 도 단위

In [ ]:
with open('skorea-provinces-geo.json',encoding = 'euc-kr') as file:
    geo3 = file.read()
    geo3 = json.loads(geo3)
    file.close()

In [ ]:
folium.GeoJson(geo3).add_to(map1)
map1

### 방문객 데이터

In [ ]:
visit=pd.read_csv('../data/외래관광객 조사_20210729.csv')
visit2 = visit.T
# visit_tot=visit.iloc[3][2:]
# visit_tot
visit2

In [ ]:
visit2.info()

### 지도에 방문객 수 표시

In [ ]:
geo3.keys()

In [ ]:
geo3

In [ ]:
len(geo3['features'])

In [ ]:
for i in range(16):
    print(geo3['features'][i]['properties']["NAME_1"] )

#### 세종 데이터가 지역 정보에 없다 -> 데이터에서도 제거

In [ ]:
# 세종 데이터 제거
visit2.drop(index='세종', inplace=True)

visit2.rename(columns={3: "전체방문객"},inplace=True)
visit2= visit2[2:]
visit2['지역']=['Seoul','Incheon','Daejeon','Daegu', 'Busan','Gwangju', 'Ulsan', \
              'Gyeonggi-do','Gangwon-do', 'Chungcheongbuk-do','Chungcheongnam-do','Jeollabuk-do', 'Jeollanam-do', \
             'Gyeongsangbuk-do','Gyeongsangnam-do', 'Jeju']
visit2

데이터 형변환

In [ ]:
visit2= visit2[["전체방문객","지역"]]

visit2 = visit2.astype({'전체방문객': 'float32'})

visit2['전체방문객']

#### log scaling

In [ ]:
import math
import numpy as np

In [ ]:
np.log2(4)

In [ ]:
visit2['방문객2']=np.log2(visit2['전체방문객'])
visit2['방문객10']=np.log10(visit2['전체방문객'])

In [ ]:
visit2

In [ ]:
choro = folium.Choropleth(geo_data = geo3,
                 data = visit2,
                 columns = ["지역","전체방문객"],
                 key_on = 'feature.properties.NAME_1',
                 legend_name='해외 방문객 비율',
                 fill_color='BuPu',
                 highlight =True).add_to(map1)

In [ ]:
choro.geojson.add_child(folium.features.GeoJsonTooltip(['NAME_1'], labels=False))

In [ ]:
map1

#### 지역별 관광객 편차가 커서 로그 스케일링 해서 시각화

In [ ]:
choro = folium.Choropleth(geo_data = geo3,
                 data = visit2,
                 columns = ["지역","방문객2"],
                 key_on = 'feature.properties.NAME_1',
                 legend_name='해외 방문객 비율(log)',
                 fill_color='BuPu',
                 highlight =True).add_to(map1)
choro.geojson.add_child(folium.features.GeoJsonTooltip(['NAME_1'], labels=False))
map1

### 국내 방문객 수 시각화

In [ ]:
#국민여행조사 _ 국내여행_20210730
visit_in=pd.read_csv('국민여행조사 _ 국내여행_20210730.csv')
visit_in2 = visit_in.T
# visit_tot=visit.iloc[3][2:]
# visit_tot
visit_in2

In [ ]:
visit_in2.drop(index='세종', inplace=True)

visit_in2.rename(columns={3: "전체방문객"},inplace=True)
visit_in2= visit_in2[2:]
visit_in2['지역']=['Seoul', 'Busan','Daegu','Incheon','Gwangju','Daejeon','Ulsan', \
              'Gyeonggi-do','Gangwon-do', 'Chungcheongbuk-do','Chungcheongnam-do','Jeollabuk-do', 'Jeollanam-do', \
             'Gyeongsangbuk-do','Gyeongsangnam-do', 'Jeju']
visit_in2

In [ ]:
visit_in2= visit_in2[["전체방문객","지역"]]

visit_in2 = visit_in2.astype({'전체방문객': 'float32'})

visit_in2['전체방문객']

In [ ]:
visit_in2

In [ ]:
choro = folium.Choropleth(geo_data = geo3,
                 data = visit_in2,
                 columns = ["지역","전체방문객"],
                 key_on = 'feature.properties.NAME_1',
                 legend_name='국내 방문객 비율',
                 fill_color='BuPu',
                 highlight =True).add_to(map1)
choro.geojson.add_child(folium.features.GeoJsonTooltip(['NAME_1'], labels=False))
map1

### 국내외 방문객수 비율 시각화

In [ ]:
#차이
visit_all= visit_in2
visit_all['국내외 방문객비율차'] =visit_in2['전체방문객']-visit2['전체방문객']
visit_all

In [ ]:
#차이
visit_all= visit_in2
visit_all['국내외 방문객비례'] =visit_in2['전체방문객']/visit2['전체방문객']# 국내 / 외국인
visit_all['국내외 방문객비례(로그)'] =np.log2(visit_in2['전체방문객']/visit2['전체방문객'])
visit_all

In [ ]:
visit_all.sort_values(by=['국내외 방문객비례'], axis=0, ascending=False)

In [ ]:
choro = folium.Choropleth(geo_data = geo3,
                 data = visit_all,
                 columns = ["지역","국내외 방문객비례(로그)"],
                 key_on = 'feature.properties.NAME_1',
                 legend_name='국내외 방문객 비례(로그2)',
                 fill_color='BuPu',
                 highlight =True).add_to(map1)
choro.geojson.add_child(folium.features.GeoJsonTooltip(['NAME_1'], labels=False))
map1